# 문제1

정규 표현식을 이용한 비밀번호 형태로 바꾸기
- 사용자에게 비밀번호를 입력 받고
- 아래 함수에 매치 시켜야할 정규 표현식을 완성하시면 됩니다!
- 어떤 문자열들 부터 바꿔야 할 지 생각해보시면 쉽게 푸실 수 있습니다.

아래 함수에 매치 시켜야할 정규 표현식을 완성하시면 됩니다!

어떤 문자열들 부터 바꿔야 할 지 생각해보시면 쉽게 푸실 수 있습니다.

In [10]:
import re
def ObjectInspector():

    string = input("비밀번호를 입력해주세요 : ")
    
    num = re.compile("[0-9]") 
    string = num.sub('3', string)
    
    kor = re.compile("[가-힣]")
    string = kor.sub('1', string)

    eng = re.compile("[a-z]", re.I)    
    string = eng.sub('2', string)

    others = re.compile("\W") #/D
    string = others.sub('4', string)
    print('encoding된 비밀번호는 ',string," 입니다.")
    
ObjectInspector()

비밀번호를 입력해주세요 : 가나다aNC123@!@
encoding된 비밀번호는  111222333444  입니다.


# 문제2

다음 소스 코드를 완성하여 주어진 이메일 주소가 올바른지 판단하도록 만드세요. emails 리스트에서 앞의 다섯 개는 올바른 형식이며 마지막 세 개는 잘못된 형식입니다.

In [14]:
import re
 
p = re.compile('^[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')
emails = ['python@mail.example.com', 'python+kr@example.com',              # 올바른 형식
          'python-dojang@example.co.kr', 'python_10@example.info',         # 올바른 형식
          'python.dojang@e-xample.com',                                    # 올바른 형식
          '@example.com', 'python@example', 'python@example-com']          # 잘못된 형식
 
for email in emails:
    print(p.match(email) != None, end=' ')

True True True True True False False False 

# 문제3

### 네이버 금융를 통한 주가 크롤링!

반도체 슈퍼싸이클과 관련한 기사를 보며 전자산업에 관심을 가지게 된 창원이는 KOSPI에 상장되어 있는 전자회사들의 주가가 궁금해졌습니다. KOSPI 상장회사 중 '전자'로 끝나는 이름을 가진 기업들의 [현재가, 전일가, 시가, 고가, 저가]를 수업 시간에 배운 csv와 html을 이용하여 크롤링해봅시다 (우선주 포함). 

1. 한국거래소는 현재 상장된 회사들의 대한 정보를 csv로 제공합니다. [http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201](http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201) 에 들어가 KOSPI 상장사에 대한 정보를 csv로 다운 받아봅시다.


2. 해당 csv를 csv 패키지를 통해 Python으로 불러옵시다. 이 때, 전자로 끝나는 기업(한글 종목약명 기준)들의 종목명을 key로, 단축코드를 value로 가지는 dictionary를 만들어주세요.
    - hint: 정규표현식 패키지인 re의 re.search 메소드를 사용해보세요!
    
    
3. 이후 네이버 금융의 해당 종목 페이지(ex. [https://finance.naver.com/item/main.nhn?code=005930](https://finance.naver.com/item/main.nhn?code=005930))에서 [종목명, 현재가, 전일가, 시가, 고가, 저가]를 크롤링하여 하나의 list로 만들어주세요. 최종적으로는 종목 별 list를 원소로 받는 'list안의 list'를 만들어야 합니다.
    - hint: html의 소스코드는 해당 페이지의 url address과 urllib.request의 urlopen 객체를 통해 가져올 수 있습니다. Ex. source = urlopen(url_address).read()
    - hint: 위에서 얻은 html 소스코드를 bs4.BeautifulSoup 객체에 사용해보세요! (parser는 lxml를 사용하시면 됩니다. Beautiful Soup의 find 함수는 'class_' 인자를 통해 class 구분이 가능합니다. Ex. find('dl', class_ = 'class_name'))
    - hint: text를 다룰 때 split 함수는 매우 유용합니다! (주가를 int가 아닌 str로 저장해주세요.)
    
    
4. 마지막으로 'list안의 list'를 미리 정의된 to_dataframe() 함수에 넣어 pandas dataframe으로 만들어봅시다.

https://www.notion.so/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2Fec3672a2-1eaf-4ad3-9f52-bee18c47a988%2FUntitled.png?table=block&id=cfd57eb1-41b5-40c7-83ee-8c247a499a00&width=930&userId=&cache=v2![image.png](attachment:image.png)


In [183]:
import pandas as pd
import csv
import re
import os
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs


def to_dataframe(lst):
    header = ['종목명','현재가', '전일가', '시가','고가','저가']
    df = pd.DataFrame(lst)
    df.columns = header
    return df


p = re.compile("전자$|전자우$") #필터링 조건 생성
stock_dict = {} #딕셔너리 생성

with open ("/Users/Jijoonghong/Downloads/stock_data.csv", "r", encoding = "cp949") as stocks:
    csv_data = csv.reader(stocks)
    for row in csv_data:
        name = row[3] #종목약명
        if p.search(name): #기준에 따라 종목약명 필터링
            stock_dict[row[3]] = row[1] #딕셔너리 추가 {종목약명 : 단축코드}

stock_num = stock_dict.values() #딕셔너리에서 코드만 추출

total_lst = [] #종목 종합용 리스트

for num in stock_num: #종목 코드내에서 반복
    lst = [] #종목별 정보용 리스트
    url = "https://finance.naver.com/item/main.nhn?code=" + num #종목별 검색
    source = urlopen(url).read() #종목별 검색
    soup = bs(source, "lxml") #lxml로 파싱
    
    name = soup.find("title").get_text() #종목명 파싱
    lst.append(name.split(":")[0]) #LG전자우 : 네이버 금융 -> LG전자우
    
    
    
    info = soup.find(class_= "rate_info") #가격정보 파싱
    result = info.find_all("span", class_ = "blind")
    # [<span class="blind">71,400</span>, <span class="blind">1,800</span>, <span class="blind">2.46</span>, 
    # <span class="blind">73,200</span>, <span class="blind">71,600</span>, <span class="blind">95,100</span>, 
    # <span class="blind">260,686</span>, <span class="blind">70,200</span>, <span class="blind">69,700</span>, 
    # <span class="blind">18,344</span>]
    
    
    lst.append(result[0].get_text()) #필요한 정보만 리스트에 추가
    lst.append(result[3].get_text())
    lst.append(result[7].get_text())
    lst.append(result[4].get_text())
    lst.append(result[8].get_text())
    total_lst.append(lst) #전체리스트에 추가

     
to_dataframe(total_lst)

,종목명,현재가,전일가,시가,고가,저가
0,LG전자우,"71,400","73,200","70,200","71,600","69,700"
1,LG전자,"146,500","154,000","148,000","150,500","146,000"
2,경인전자,"30,000","31,550","30,700","31,400","29,150"
3,광전자,"2,435","2,525","2,460","2,495","2,420"
4,대덕전자,"14,600","14,850","14,250","14,800","14,150"
5,대동전자,"5,540","5,650","5,580","5,650","5,440"
6,덕우전자,"9,060","9,380","8,990","9,200","8,840"
7,모베이스전자,"2,655","2,640","2,510","2,695","2,510"
8,바른전자,"1,005",0,0,0,0
9,빛샘전자,"6,420","6,710","6,640","6,640","6,330"


# 문제4

### 네이버 뉴스 기사 제목을 크롤링 해보자!

html구조와 csv 파일 쓰는 방법을 활용하여 원하는 검색어로 검색했을 때 나오는 뉴스 기사 제목들 관련도 순으로 저장해봅시다.

민진이는 삼성전자 92층 펜트 하우스에 거주하고 있습니다. 연이은 하락세로 삼성전자에 나쁜 소식이 있나 궁금했던 민진이는 네이버에서 '삼성전자'라는 검색어를 입력해서 관련 기사 제목들을 모아 삼성전자의 소식을 살펴보려고 합니다.  원하는 검색어, 원하는 페이지를 입력했을 때 뉴스 기사의 페이지, 제목을 파일에 저장 해주는 웹크롤러를 만들어 봅시다!

- 페이지 당 큰 기사 제목 10개씩 크롤링 하면 됩니다.
- 시작 페이지와 종료 페이지를 입력 했을 때, 만일 시작 페이지가 종료페이지보다 큰 수라면 '페이지 설정을 다시 해주세요.'라는 말과 함께 다시 페이지를 입력할 수 있도록 main() 함수를 다시 실행해주세요.
- 제가 제공한 틀을 써도 되고, 틀부터 새로 코드를 작성하셔도 괜찮습니다.
- pandas 같은 경우는 main 함수에서 저장한 파일을 여는 수단으로만 사용됩니다. 퀴즈 작성하실 땐 전혀 상관 없습니다!
- BeatuifulSoup에 대해 잘 모르신다면 구글에 '네이버 크롤링'을 검색하면 다양한 예시들이 나와있는데, 이를 참고해서 코드를 작성해주세요.
- Beautifulsoup의 get_text() method를 사용하면 편리합니다.

In [184]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re, time, os, csv

def main():
    print('#'*100)
    print('관련도 순으로 검색어에 대한 기사 제목을 저장하려고 합니다.')
    print('원하는 검색어와 페이지를 입력해주세요.')
    print('#'*100)
    query = input('검색어 : ')
    start_page = int(input('시작 페이지 : '))
    end_page = int(input('끝 페이지 : '))
    
    if end_page - start_page < 0:
        print("페이지 설정을 다시 해주세요")
        main()
        
    else:
        print('#'*100)
        start_time = time.time()
        save_file_path = r'/Users/jijoonghong/Downloads/'
        os.chdir(save_file_path)
        save_file_name = 'naver_crawling.csv'
        with  open(save_file_name, 'w', newline='', encoding='cp949') as f:
            cw = csv.writer(f)
            cw.writerow(['title','page'])
            crawling(cw, query, start_page, end_page)
            tmp_running_time = time.time() - start_time
            print('%s\ntmp running time : %d m  %0.2f s\n%s\n'%('#'*100, tmp_running_time//60, tmp_running_time%60, '#'*100))
        print('{0}\n!!!!COMPLETE!!!!\n{0}'.format('#'*100))
        crawling_data = pd.read_csv(save_file_name, encoding='cp949')
        print(crawling_data.shape)
        print(crawling_data.tail())
        

def crawling(cw, query, start_page, end_page):
    l = [] #행으로 사용할 리스트
    curr_page = start_page #현재 페이지 지정

    for i in range(end_page-start_page+1):
        idx = (curr_page-1)*10 + 1 #1, 11, 21, 31, 41.....
        page = "?&where=news&query="+query+"&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=63&start="+str(idx)
        url = 'https://search.naver.com/search.naver'+page
        r = requests.get(url)
        
        soup = bs(r.text, 'html.parser') #html 파싱
        sources = soup.select('div.group_news > ul.list_news > li div.news_area > a') #상위에서 하위 범주로 이동
        
        print(sources)
        
        for source in sources :
            l = [source.attrs['title'], curr_page] #html 속성 지정
            cw.writerow(l) #csv에 내용 추가
        curr_page += 1 #페이지 증가

main()



####################################################################################################
관련도 순으로 검색어에 대한 기사 제목을 저장하려고 합니다.
원하는 검색어와 페이지를 입력해주세요.
####################################################################################################
검색어 : 2
시작 페이지 : 42
끝 페이지 : 2
페이지 설정을 다시 해주세요
####################################################################################################
관련도 순으로 검색어에 대한 기사 제목을 저장하려고 합니다.
원하는 검색어와 페이지를 입력해주세요.
####################################################################################################
검색어 : 삼성전자
시작 페이지 : 2
끝 페이지 : 5
####################################################################################################
[<a class="news_tit" href="https://www.hankyung.com/economy/article/202102278705i" onclick="return goOtherCR(this, 'a=nws*h.tit&amp;r=64&amp;i=88000107_000000000000000004505758&amp;g=015.0004505758&amp;u='+urlencode(this.href));" target="_blank" title="&quot;반도체 생산의 핵심&quot;…삼성이 애타게 찾는 '이것' 뭐길래 [황정수의 반도체

[<a class="news_tit" href="http://news.mt.co.kr/mtview.php?no=2021022410343424374" onclick="return goOtherCR(this, 'a=nws*h.tit&amp;r=64&amp;i=880000C2_000000000000000004548073&amp;g=008.0004548073&amp;u='+urlencode(this.href));" target="_blank" title="또 한번 삼성의 초격차…데이터센터 전용 낸드, 페이스북 공급">또 한번 <mark>삼성</mark>의 초격차…데이터센터 전용 낸드, 페이스북 공급</a>, <a class="news_tit" href="https://view.asiae.co.kr/article/2021022702494003417" onclick="return goOtherCR(this, 'a=nws*j.tit&amp;r=66&amp;i=88000385_000000000000000004855844&amp;g=277.0004855844&amp;u='+urlencode(this.href));" target="_blank" title="[르포]여의도 '더현대 서울'서 가전 정면승부…삼성·LG 매장 가보니">[르포]여의도 '더현대 서울'서 가전 정면승부…<mark>삼성</mark>·LG 매장 가보니</a>, <a class="news_tit" href="https://view.asiae.co.kr/article/2021022609452105063" onclick="return goOtherCR(this, 'a=nws*e.tit&amp;r=67&amp;i=88000385_000000000000000004855368&amp;g=277.0004855368&amp;u='+urlencode(this.href));" target="_blank" title="카카오, 액면분할에 주가도 강세…제2의 삼성전자·네이버?">카카오, 액면분할에 주가도 강세…제2의 <mark>삼성